In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os

In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [3]:
def calculate_angle(a,b,c):
    # a is first angle, b is second angle, c is 3rd angle
    a = np.array(a)
    b = np.array(b)
    c = np.array(b)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0/np.pi)
    if (angle > 180):
        return 360-angle;
    return angle

In [4]:
# nose 0, left eye 1, right eye 2, left ear 3, right ear 4, 
# left shoulder 5, right shoulder 6, left elbow 7, right elbow 8, 
# left wrist 9, right wrist 10, left hip 11, right hip 12, left knee 13, right knee 14, left ankle 15, right ankle 16
# elbow, shoulder, hip, ankle

In [5]:
EDGES = {
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    #y, x, c = frame.shape
    #shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = keypoints[p1-5]
        y2, x2, c2 = keypoints[p2-5]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [7]:
"""def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1])) # confidence metric c, dont transform that
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) #.circle creates circle, 4 is size, 255 is color green, -1 is fill circle
            """
def draw_keypoints(frame, keypoints, confidence_threshold):
    for kp in keypoints:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1)

In [8]:
def get_affine_transform_to_fixed_sizes_with_padding(size, new_sizes):
    width, height = new_sizes
    scale = min(height / float(size[1]), width / float(size[0]))
    M = np.float32([[scale, 0, 0], [0, scale, 0]])
    M[0][2] = (width - scale * size[0]) / 2
    M[1][2] = (height - scale * size[1]) / 2
    return M

In [10]:
def test():
    l_one = [] # l = the last frame recorded
    l_l_one = [] # the last last frame recorded
    went_down_squat = False
    went_down_jacks = False
    squat_count = 0
    jj_count = 0
    frame_idx = 0 
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        # take frame from open cv video capture
        ret, frame = cap.read()
        img = frame.copy()
        #start of taking coordinates
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis = 0), 192, 192)
        input_image = tf.cast(img, dtype=tf.float32)
        
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]["index"])[0, 0]
        # end of taking coordinates
        
        orig_w, orig_h = frame.shape[:2]
        M = get_affine_transform_to_fixed_sizes_with_padding((orig_w, orig_h), (192, 192))
        # M has shape 2x3 but we need square matrix when finding an inverse
        M = np.vstack((M, [0, 0, 1]))
        M_inv = np.linalg.inv(M)[:2]
        xy_keypoints = keypoints_with_scores[:, :2] * 192
        xy_keypoints = cv2.transform(np.array([xy_keypoints]), M_inv)[0]
        keypoints_with_scores = np.hstack((xy_keypoints, keypoints_with_scores[:, 2:]))

        # 5,6 shoulder; 7,8 elbow; 11,12 hip; 13,14 knee; 15,16 ankle
        # sectioning off all scores were interested in
        wanted_scores = keypoints_with_scores[5:]
        
        go = False
        
        for i in range(11):
            if (i == 2 or i == 3 or i == 4 or i == 5):
                continue
            if (wanted_scores[i][2] < 0.15):
                go = True
                break
        if (go == False):
            # all angles for hands - 1st left shoulder, hip, knee
            back_angle = [calculate_angle(wanted_scores[0], wanted_scores[6], wanted_scores[8])] + [calculate_angle(wanted_scores[1], wanted_scores[7], wanted_scores[9])]
            # all angles for legs - 1st left hip, knee, ankle
            knee_angle = [calculate_angle(wanted_scores[6], wanted_scores[8], wanted_scores[10])] + [calculate_angle(wanted_scores[1], wanted_scores[7], wanted_scores[11])]
            
            underarm_angle = [calculate_angle(wanted_scores[2], wanted_scores[0], wanted_scores[6])] + [calculate_angle(wanted_scores[3], wanted_scores[1], wanted_scores[7])]
            leg_angle = [calculate_angle(wanted_scores[0], wanted_scores[6], wanted_scores[10])] + [calculate_angle(wanted_scores[1], wanted_scores[7], wanted_scores[11])]

            if (went_down_squat and ((back_angle[0] > 170 and knee_angle[0] > 170) or (back_angle[1] > 170 and knee_angle[1] > 170))):
                squat_count += 1
                went_down_squat = False
            elif (went_down_squat == False and ((back_angle[0] < 145 and knee_angle[0] <= 120) or (back_angle[1] < 145 and knee_angle[1] <= 120))):
                went_down_squat = True
                
            if (went_down_jacks and underarm_angle[0] > 130):
                jj_count += 1
                went_down_jacks = False
                # print(jj_count)
            elif (went_down_jacks == False and underarm_angle[0] < 30):
                went_down_jacks = True
        
        full_text_for_jj = "Jumping Jacks: " + str(jj_count)
        full_text_for_squat = "Squats: " + str(squat_count)
        
        cv2.putText(frame,  
                full_text_for_jj,  
                (50, 50),  
                cv2.FONT_HERSHEY_SIMPLEX, 1,  
                (0, 0, 0),  
                3,  
                cv2.LINE_4)
        cv2.putText(frame,  
                full_text_for_squat,  
                (50, 100),  
                cv2.FONT_HERSHEY_SIMPLEX, 1,  
                (0, 0, 0),  
                3,  
                cv2.LINE_4)
        draw_keypoints(frame, wanted_scores, 0.4)
        draw_connections(frame, wanted_scores, EDGES, 0.4)
        cv2.imshow('Squats', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
    # h_avg_up = h_angle_up / jj_count
    # h_avg_down = h_angle_down / jj_count
    # l_avg_up = l_angle_up / jj_count
    # l_avg_down = l_angle_down / jj_count
    return #h_avg_up, h_avg_down, l_avg_up, l_avg_down

In [11]:
test()